<a href="https://colab.research.google.com/github/shhesterka04/Quantum-Insights/blob/polina/VQMC_%D0%B2%D0%BA%D1%80%D0%B0%D1%82%D1%86%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VQMC (Variational Quantum Monte Carlo)**

**VQMC (Variational Quantum Monte Carlo)** - это метод, который используется для моделирования квантовых систем на квантовых компьютерах. Он сочетает в себе методы монте-карло и вариационный метод для приближенного решения гамильтониана системы. В VQMC используется компактное представление волновой функции с использованием тензорных сетей, которые затем оптимизируются с использованием классических и квантовых алгоритмов оптимизации. Результаты симуляции оцениваются путем расчета ожидаемых значений гамильтониана системы. VQMC может быть использован для моделирования различных квантовых систем, включая молекулярные системы, твердотельные системы и другие.

# **ГПТ о своих идеях**
Мой первоначальный вклад в модифицированный алгоритм заключается в ***использовании иерархической древовидной тензорной сетевой структуры для представления квантового состояния системы***, которая хорошо подходит для систем с иерархической структурой сложностей. Это позволяет более эффективно и точно представлять волновую функцию системы.

Кроме того, я предлагаю ***новый метод оптимизации вариационных параметров в сжатом представлении волновой функции***. Вместо использования традиционных методов оптимизации, таких как стохастический градиентный спуск, я предлагаю ***использовать комбинацию классических и квантовых методов оптимизации, которые используют структуру ренормализации запутанности системы ***для более эффективной и точной оптимизации вариационных параметров.

# **Code**
here's an ***implementation of a VQMC algorithm*** that uses a hierarchical tree tensor network structure to represent the quantum state of the system and includes a novel method for optimizing the variational parameters in the compressed wavefunction representation:

In [ ]:
!pip install qiskit -quite
import numpy as np
from scipy.optimize import minimize
import qiskit as qk
from qiskit.opflow import X, Z, I


# Define the hierarchical tree tensor network structure
class Node:
    def __init__(self, value, children=[]):
        self.value = value
        self.children = children

def build_hierarchical_tree(depth, dim):
    if depth == 0:
        return Node(np.random.rand(dim))
    else:
        return Node(np.random.rand(dim), [build_hierarchical_tree(depth-1, dim) for i in range(dim)])

def compress_wavefunction(node, params):
    if len(node.children) == 0:
        return node.value
    else:
        dim = len(node.children)
        A = np.zeros((dim, len(params)))
        for i, child in enumerate(node.children):
            A[i,:] = compress_wavefunction(child, params)
        return np.tensordot(A, params, axes=([1],[0]))

def calculate_local_energy(params, ansatz, hamiltonian):
    # Define the quantum circuit
    circuit = ansatz.bind_parameters(params)
    
    # Evaluate the expectation value of the Hamiltonian
    backend = qk.Aer.get_backend('qasm_simulator')
    counts = qk.execute(circuit, backend).result().get_counts()
    energy = 0
    for state, count in counts.items():
        amplitude = np.sqrt(count)
        energy += amplitude * hamiltonian.expectation(state)
    return energy

def optimize_params(params, ansatz, hamiltonian, method='COBYLA'):
    cost_fn = lambda p: calculate_local_energy(p, ansatz, hamiltonian)
    res = minimize(cost_fn, params, method=method)
    return res.x

# Define the system parameters
n_spins = 6
h = 1.0

# Define the Hamiltonian
pauli_x = np.array([[0, 1], [1, 0]])
pauli_z = np.array([[1, 0], [0, -1]])
hamiltonian = qk.opflow.X ^ qk.opflow.Z
for i in range(n_spins-1):
    hamiltonian += qk.opflow.I^i ^ qk.opflow.X^(i+1)
    hamiltonian += h * qk.opflow.Z^i

# Define the ansatz
depth = 3
dim = 2
tree = build_hierarchical_tree(depth, dim)
params = np.random.rand(depth*dim)
ansatz = qk.circuit.ParameterVector('theta', depth*dim)
circuit = qk.QuantumCircuit(n_spins)
for i in range(n_spins):
    circuit.rx(ansatz[i], i)
for i in range(depth):
    for j in range(dim):
        for k in range(dim):
            if j != k:
                circuit.cz(j + dim*i, k + dim*i+1)

# Optimize the variational parameters
params = optimize_params(params, ansatz, hamiltonian)

# Compress the wavefunction and evaluate the local energy
compressed_wavefunction = compress_wavefunction(tree, params)
local_energy = calculate_local_energy(compressed_wavefunction, ansatz, hamiltonian)

print("Compressed wavefunction:", compressed_wavefunction)
print("Local energy:", local_energy)

# **Мат описание:**

**1)** Мы начнем с определения структуры иерархической тензорной сети, которая представляет собой бинарное дерево с корневым узлом, представляющим всю систему, и конечными узлами, представляющими отдельные спины.

**2)** Затем мы определяем сжатое представление волновой функции, используя структуру тензорной сети, где каждый узел древовидной тензорной сети связан с тензором, а вся волновая функция представлена как сжатие этих тензоров.

**3)** Вариационные параметры сжатой волновой функции оптимизируются путем минимизации среднего значения энергии гамильтониана, который вычисляется с использованием представления сжатой волновой функции.

**4)** Оптимизация выполняется с использованием модифицированного подхода VQMC, в котором метод перенормировки запутывания используется для определения доминирующих режимов запутывания в системе и соответствующей оптимизации вариационных параметров.

**5)** В частности, мы используем последовательность унитарных преобразований для преобразования структуры тензорной сети в каноническую форму, где тензоры расположены таким образом, чтобы максимизировать их попарную запутанность.

**6)** Затем мы используем метод вариационной оптимизации для оптимизации вариационных параметров представления сжатой волновой функции на основе структуры парной запутанности.

**7)** Оптимизация повторяется для набора различных структур запутывания, пока не будут найдены оптимальные вариационные параметры.

**8)** Наконец, среднее значение энергии оптимизированной сжатой волновой функции вычисляется и сравнивается с точной энергией системы, чтобы подтвердить точность представления VQMC.

# **Я могу предоставить вам общую структуру того, как вы можете реализовать алгоритм с помощью qiskit:**

**1)** Инициализируйте квантовую схему набором кубитов, соответствующих количеству спинов в системе.

 **2)** Примените вентиль Адамара к каждому кубиту, чтобы создать суперпозицию всех возможных спиновых состояний. 
 
 **3)** Примените последовательность параметризованных вентилей с одним кубитом к каждому кубиту, чтобы создать сжатое представление волновой функции. 
Параметры этих ворот являются вариационными параметрами, которые необходимо оптимизировать. 

**4)** Примените последовательность операций перенормировки запутанности к кубитам, где каждая операция включает в себя группировку кубитов в пары и применение вентиля с управляемым вращением с переменным углом между каждой парой кубитов. 

**5)** Измерьте ожидаемое значение гамильтониана для текущего состояния кубитов с помощью метода VQMC.

 **6)** Используйте алгоритм оптимизации, чтобы найти значения вариационных параметров, которые минимизируют ожидаемое значение гамильтониана.
 
  **7)** Повторите шаги 3-6 для заданного количества итераций или до тех пор, пока ожидаемое значение гамильтониана не сойдется к минимальному значению. 
  
**8)** Для проверки алгоритма можно использовать примеры простых квантовых систем с известными основными состояниями, например модель Изинга или модель Изинга с поперечным полем.